### LangGraph Agent - Customer Support multivoice Agent

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_community.vectorstores import PGVector



In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import PGVector

system = """You are Andrea, a knowledgeable and friendly assistant in a telecommunications company. Your expertise lies in various mobile plans and upgrades. Your role is to help users understand their options and assist them with their queries about mobile plans and upgrades. Always respond in a helpful and professional manner.
Always speak to the user with his name.

Username: {username}

Remember, your goal is to make the user feel supported and informed. Always be courteous and clear in your responses.
"""
prompt_template = PromptTemplate.from_template(system)

In [4]:
import requests
from langchain_core.tools import tool
from typing import Optional
import time

API_URL = "http://127.0.0.1:8000"
ADMIN_USERNAME = "admin1"
ADMIN_PASSWORD = "admin1password"

categories = ["basic", "normal", "premium"]

def login():
    login_response = requests.post(f"{API_URL}/token/", data={"username": ADMIN_USERNAME, "password": ADMIN_PASSWORD})
    if login_response.status_code != 200:
        print(f"Login failed: {login_response.json().get('detail')}")
        return None, f"Login failed: {login_response.json().get('detail')}"
    access_token = login_response.json().get("access_token")
    headers = {"Authorization": f"Bearer {access_token}"}
    print("Login successful, headers obtained")
    return headers, None

def ask_admin(action: str, username: str, category: Optional[str] = None):
    try:
        headers, error = login()
        if error:
            return None, error

        ask_data = {"action": action, "username": username}
        if category:
            ask_data["category"] = category

        print(f"Requesting admin approval with data: {ask_data}")
        response = requests.post(f"{API_URL}/ask_admin/", json=ask_data, headers=headers)
        if response.status_code != 200:
            print(f"Failed to request admin approval: {response.json().get('detail')}")
            return None, f"Failed to request admin approval: {response.json().get('detail')}"

        print("Admin approval requested")
        return "Admin approval requested", None
    except Exception as e:
        print(f"Failed to execute ask_admin. Error: {repr(e)}")
        return None, f"Failed to execute. Error: {repr(e)}"

def wait_for_admin_approval(action: str, username: str, category: str = None):
    print("Waiting for admin approval...")
    while True:
        response = requests.get(f"{API_URL}/check_confirmation/{username}")
        if response.status_code == 200:
            result = response.json()
            print(f"Received admin approval response: {result}")
            message = result.get("message")
            # Exit loop if a final decision has been made
            if message in ["Admin denied the request", "Contract created", "Contract will be cancelled in 3 months"]:
                return result
        time.sleep(2)  # Add a delay before retrying to avoid spamming the server


@tool
def create_contract_tool(username: str, category: str):
    """
    Create a new contract for a user with a specific category.

    Args:
        username (str): Username of the user for whom the contract is being created.
        category (str): Category of the contract. Must be one of "basic", "normal", or "premium".

    Returns:
        str: A string indicating the result of the admin approval process and contract creation.
    """
    print(f"Starting contract creation for user: {username}, category: {category}")

    # Step 0: Check if the user already has a contract
    headers, error = login()
    if error:
        print(f"Error during login: {error}")
        return error

    print(f"Fetching contract details for username: {username}")
    user_contract_response = requests.get(f"{API_URL}/contracts/user/{username}", headers=headers)
    if user_contract_response.status_code == 200:
        user_contract = user_contract_response.json()
        print(f"User contract details: {user_contract}")
        # Check if the user has a valid contract category
        if user_contract.get('category') in categories:
            return f"User already has a contract: {user_contract}"
        else:
            print("No valid contract found for the user.")
    elif user_contract_response.status_code == 404:
        print("No contract found for the user.")
    else:
        print(f"Failed to fetch user contract details: {user_contract_response.json().get('detail')}")
        return f"Failed to fetch user contract details: {user_contract_response.json().get('detail')}"

    # Step 1: Request admin approval
    admin_request, error = ask_admin("create", username, category)
    if error:
        print(f"Error during admin approval request: {error}")
        return error

    # Inform that admin approval is requested
    if admin_request == "Admin approval requested":
        # Wait for admin approval
        approval_result = wait_for_admin_approval("create", username, category)
        print("APPROVAL RESULT: ", approval_result)

        if approval_result.get("message") == "Admin denied the request":
            print(f"Admin denied the request: {approval_result.get('message')}")
            return approval_result.get('message')
        elif approval_result.get("message") == "Contract created":
            print(f"Admin created the contract: {approval_result}")
            return f"Contract successfully created: ID {approval_result['id']}, Category {approval_result['category']}, Contract Time {approval_result['contract_time']}, User ID {approval_result['user_id']}"

    return "Unexpected flow reached"

@tool
def delete_contract_tool(username: str):
    """
    Delete the contract for a user, requiring admin approval.

    Args:
        username (str): Username of the user whose contract is being deleted.

    Returns:
        str: A string indicating the result of the admin approval process and contract deletion.
    """
    print(f"Starting contract deletion for user: {username}")

    headers, error = login()
    if error:
        print(f"Error during login: {error}")
        return error

    print(f"Fetching contract details for username: {username}")
    user_contract_response = requests.get(f"{API_URL}/contracts/user/{username}", headers=headers)
    if user_contract_response.status_code != 200:
        return f"Failed to fetch user contract details: {user_contract_response.json().get('detail')}"

    admin_request, error = ask_admin("delete", username)
    if error:
        print(f"Error during admin approval request: {error}")
        return error

    if admin_request == "Admin approval requested":
        approval_result = wait_for_admin_approval("delete", username)
        print("APPROVAL RESULT: ", approval_result)

        if approval_result.get("message") == "Admin denied the request":
            print(f"Admin denied the request: {approval_result.get('message')}")
            return approval_result.get('message')
        elif approval_result.get("message") == "Contract will be cancelled in 3 months":
            print(f"Admin approved the contract deletion: {approval_result}")
            return f"Contract will be cancelled in 3 months"

    return "Unexpected flow reached"

@tool
def update_contract_tool(username: str, new_category: str):
    """
    Update the contract category for a user.

    Args:
        username (str): Username of the user whose contract is being updated.
        new_category (str): New category of the contract. Must be different from the current category and one of "basic", "normal", or "premium".

    Returns:
        str: A string indicating the result of the contract update.
    """
    print(f"Starting contract update for user: {username}, new category: {new_category}")

    headers, error = login()
    if error:
        print(f"Error during login: {error}")
        return error

    print(f"Fetching contract details for username: {username}")
    user_contract_response = requests.get(f"{API_URL}/contracts/user/{username}", headers=headers)
    if user_contract_response.status_code != 200:
        return f"Failed to fetch user contract details: {user_contract_response.json().get('detail')}"

    user_contract = user_contract_response.json()
    if user_contract.get('category') == new_category:
        return f"New category must be different from the current category: {user_contract['category']}"

    update_data = {"category": new_category}
    response = requests.put(f"{API_URL}/contracts/{user_contract['id']}/", json=update_data, headers=headers)
    if response.status_code != 200:
        return f"Failed to update contract: {response.json().get('detail')}"

    updated_contract = response.json()
    return f"Contract successfully updated to: {updated_contract['category']}"

tools = [create_contract_tool, delete_contract_tool, update_contract_tool]

In [5]:
from langchain_core.messages.human import HumanMessage
from langchain_core.messages.system import SystemMessage

sys_msg = [SystemMessage(content=prompt_template.format(username="aaaa"))]
hu_msg = [HumanMessage(content="I would like to delete my contract.")]

chat_history = []

messages = sys_msg + chat_history + hu_msg
model = ChatOpenAI(model="gpt-4o-mini")
model_with_tools = model.bind_tools(tools=tools)

result = model_with_tools.invoke(messages)

In [6]:
messages.append(result)

In [7]:
result.tool_calls

[{'name': 'delete_contract_tool',
  'args': {'username': 'aaaa'},
  'id': 'call_ZSGQhNFn2qTTIcCowJ54BeDc',
  'type': 'tool_call'}]

In [8]:
from langchain_core.messages import ToolMessage

for tool_call in result.tool_calls:
    print("Use Tool:", tool_call)
    selected_tool = {tool.name.lower(): tool for tool in tools}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    print(tool_output)
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

Use Tool: {'name': 'delete_contract_tool', 'args': {'username': 'aaaa'}, 'id': 'call_ZSGQhNFn2qTTIcCowJ54BeDc', 'type': 'tool_call'}
Starting contract deletion for user: aaaa
Login successful, headers obtained
Fetching contract details for username: aaaa
Login successful, headers obtained
Requesting admin approval with data: {'action': 'delete', 'username': 'aaaa'}
Admin approval requested
Waiting for admin approval...
Received admin approval response: {'message': 'Waiting for admin confirmation'}
Received admin approval response: {'message': 'Waiting for admin confirmation'}
Received admin approval response: {'message': 'Waiting for admin confirmation'}
Received admin approval response: {'message': 'Contract will be cancelled in 3 months', 'id': 1, 'category': 'premium', 'contract_time': '2024-10-27T19:03:13.409328', 'user_id': 3}
Received admin approval response: {'message': 'Waiting for admin confirmation'}
Received admin approval response: {'message': 'Waiting for admin confirmatio

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /check_confirmation/aaaa (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002082BC771D0>: Failed to establish a new connection: [WinError 10061] Es konnte keine Verbindung hergestellt werden, da der Zielcomputer die Verbindung verweigerte'))

In [ ]:
messages

In [ ]:
model_with_tools.invoke(messages)